# <center>第二章 基础数值计算方法</center>
### <center>许文立</center>
### <center>SFU/AHU/CIMERS/国民经济工程实验室(北京)</center>
## 2.1 概要

* 了解计算经济学中基础的一些数值计算技术
* 掌握数值计算在Julia中的应用

## 2.2 线性代数回顾
定义一个线性方程：n个未知数，n个系数
$$a_1 x_1 +a_2 x_2 +……+a_n x_n =b$$
所有变量都是一次，且没有非线性函数。多个线性方程组成的系统就是线性方程组：一种紧凑的方式表达线性方程组：
$$Ax=b$$
其中，A是一个$n \times n$的系数矩阵，元素为$a_{i,j}$，b为n维向量，x为n为向量。

两类解法：
* 直接方法
* 迭代方法
